In [11]:
# Path to where deeplearningproject is stored on drive that contains the data and the git repo (may have to be changed depending on user)
deep_learn_drive_path = '/content/drive/MyDrive/CS 6955/deeplearningproject/'
# To use with colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os

Mounted at /content/drive


In [12]:
# Clone repo if needed
os.chdir(deep_learn_drive_path)
!git clone https://github.com/dalbyryan3/cs-6955-deep-learning-final-project.git

Cloning into 'cs-6955-deep-learning-final-project'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [19]:
# cd into git repo
os.chdir(deep_learn_drive_path + 'cs-6955-deep-learning-final-project')
print('Git directory content: {0}'.format(os.listdir()))

Git directory content: ['.git', 'README.md']
